In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler

# 그래프 외부에 출력
%matplotlib tk

tf.set_random_seed(777)  # 어느 컴퓨터에서 이 코드를 실행해도 학습 방향이 같도록, 다시 수행해도 같도록

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
# train Parameters
seq_length = 88
data_dim = 4
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 772

scaler = MinMaxScaler(feature_range=(0,1)) # 데이터 일반화
scaler2 = MinMaxScaler(feature_range=(0, 1)) # 데이터 일반화

# xy = np.loadtxt('./v3data/train_v3_data_cct.csv', delimiter=',')
# cct, cas_swr, 446to477, uvb, temp, ptmt (892행 5열)
xy = np.loadtxt('./v5data/train_v5_data.csv', delimiter=',')
x = scaler.fit_transform(xy[:, [1, 2, 3,4]]) # x = 맨 마지막 ptmt 제외 모든 것
y = scaler2.fit_transform(xy[:, [0]])  # y = ptmt


# build a dataset
dataX = []
dataY = []

# print(len(x))
# print(len(y))
# print(x[0])
# print(y[0])
# print(xy[:, [0,2,3,4]])

for i in range(0, len(x) - seq_length): # 한 행씩 dataX, Y에 추가
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    dataX.append(_x)
    dataY.append(_y)
#     print(np.shape(_x))


data = '2018-04-21'
# xy2 = np.loadtxt('./v3data/'+ data +'.csv',delimiter=',')
# xy2 = np.loadtxt('./v3data/test_v3_data_cct_180331.csv', delimiter=',')
xy2 = np.loadtxt('./v5data/test_v5_data_20180421.csv', delimiter=',') # 예측할 날짜

# x2 = scaler.fit_transform(xy2) # 데이터 없을때 (데이터란 ptmt 값)
x2 = scaler.fit_transform(xy2[:, [1, 2, 3,4]]) # 데이터 있을때
y2 = scaler2.fit_transform(xy2[:, [0]]) #데이터 있을때

# print(np.shape(x2))

dataX2 = []
dataY2 = [] #데이터 있을때

for i in range(0, len(x2) - seq_length):
    _x2 = x2[i:i+seq_length]
    _y2 = y2[i+seq_length] #데이터 있을때
    dataX2.append(_x2)
    dataY2.append(_y2) #데이터 있을때
# train/test split

train_size = len(dataX)
test_size = len(dataX2)

# print(train_size)
# print(test_size)

trainX, testX = np.array(dataX[0:train_size]),np.array(dataX2[0:test_size])
trainY = np.array(dataY[0:train_size])

# print(np.shape(trainX))

In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)
    dataY2 = scaler2.inverse_transform(dataY2) # 데이터 있을때

[step: 0] loss: 4582.3193359375
[step: 1] loss: 614.9935302734375
[step: 2] loss: 431.81109619140625
[step: 3] loss: 1180.817138671875
[step: 4] loss: 1155.3004150390625
[step: 5] loss: 677.7159423828125
[step: 6] loss: 271.30908203125
[step: 7] loss: 147.4420166015625
[step: 8] loss: 257.4156494140625
[step: 9] loss: 417.9058837890625
[step: 10] loss: 483.04815673828125
[step: 11] loss: 429.60986328125
[step: 12] loss: 310.45574951171875
[step: 13] loss: 192.8248291015625
[step: 14] loss: 122.57412719726562
[step: 15] loss: 112.38102722167969
[step: 16] loss: 146.35794067382812
[step: 17] loss: 194.17166137695312
[step: 18] loss: 227.44107055664062
[step: 19] loss: 231.3511199951172
[step: 20] loss: 207.54429626464844
[step: 21] loss: 169.3095703125
[step: 22] loss: 133.29443359375
[step: 23] loss: 111.99490356445312
[step: 24] loss: 109.5997085571289
[step: 25] loss: 121.92073822021484
[step: 26] loss: 139.6439208984375
[step: 27] loss: 153.08035278320312
[step: 28] loss: 156.2369079

[step: 230] loss: 98.30979919433594
[step: 231] loss: 98.29946899414062
[step: 232] loss: 98.28910827636719
[step: 233] loss: 98.27880096435547
[step: 234] loss: 98.26840209960938
[step: 235] loss: 98.2580795288086
[step: 236] loss: 98.24769592285156
[step: 237] loss: 98.23733520507812
[step: 238] loss: 98.22693634033203
[step: 239] loss: 98.21649169921875
[step: 240] loss: 98.20613098144531
[step: 241] loss: 98.19560241699219
[step: 242] loss: 98.18521118164062
[step: 243] loss: 98.17471313476562
[step: 244] loss: 98.16419982910156
[step: 245] loss: 98.15371704101562
[step: 246] loss: 98.14324951171875
[step: 247] loss: 98.13276672363281
[step: 248] loss: 98.1221923828125
[step: 249] loss: 98.111572265625
[step: 250] loss: 98.10105895996094
[step: 251] loss: 98.09046936035156
[step: 252] loss: 98.07982635498047
[step: 253] loss: 98.0692138671875
[step: 254] loss: 98.05860900878906
[step: 255] loss: 98.04791259765625
[step: 256] loss: 98.03724670410156
[step: 257] loss: 98.026550292968

[step: 460] loss: 95.07431030273438
[step: 461] loss: 95.05268859863281
[step: 462] loss: 95.03080749511719
[step: 463] loss: 95.00889587402344
[step: 464] loss: 94.98686218261719
[step: 465] loss: 94.96467590332031
[step: 466] loss: 94.94242858886719
[step: 467] loss: 94.91999816894531
[step: 468] loss: 94.89743041992188
[step: 469] loss: 94.87480163574219
[step: 470] loss: 94.85193634033203
[step: 471] loss: 94.82904815673828
[step: 472] loss: 94.8060302734375
[step: 473] loss: 94.78279113769531
[step: 474] loss: 94.75946044921875
[step: 475] loss: 94.73599243164062
[step: 476] loss: 94.71240234375
[step: 477] loss: 94.68865966796875
[step: 478] loss: 94.66475677490234
[step: 479] loss: 94.64070129394531
[step: 480] loss: 94.61654663085938
[step: 481] loss: 94.59220886230469
[step: 482] loss: 94.56787109375
[step: 483] loss: 94.54322814941406
[step: 484] loss: 94.51840209960938
[step: 485] loss: 94.4935531616211
[step: 486] loss: 94.46855163574219
[step: 487] loss: 94.44329833984375


[step: 689] loss: 84.3709487915039
[step: 690] loss: 84.35496520996094
[step: 691] loss: 84.33908081054688
[step: 692] loss: 84.3232650756836
[step: 693] loss: 84.30751037597656
[step: 694] loss: 84.29188537597656
[step: 695] loss: 84.27622985839844
[step: 696] loss: 84.26071166992188
[step: 697] loss: 84.24526977539062
[step: 698] loss: 84.22987365722656
[step: 699] loss: 84.21464538574219
[step: 700] loss: 84.19947052001953
[step: 701] loss: 84.18438720703125
[step: 702] loss: 84.16932678222656
[step: 703] loss: 84.15435791015625
[step: 704] loss: 84.13954162597656
[step: 705] loss: 84.12477111816406
[step: 706] loss: 84.11009979248047
[step: 707] loss: 84.09549713134766
[step: 708] loss: 84.08106231689453
[step: 709] loss: 84.06666564941406
[step: 710] loss: 84.0523910522461
[step: 711] loss: 84.03821563720703
[step: 712] loss: 84.02406311035156
[step: 713] loss: 84.01007843017578
[step: 714] loss: 83.99614715576172
[step: 715] loss: 83.9823226928711
[step: 716] loss: 83.96852111816

In [1]:
# Plot predictions
plt.title(data + ' predict')
plt.plot(dataY2, label='raw data') # 데이터 있을때
plt.plot(test_predict, label='predict')
plt.xlabel("Time Period")
# plt.ylabel("CAS_SWR")
# plt.ylabel("photometric")
plt.ylabel("cct")
plt.xticks(np.arange(0, 772, step = 56), ['6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7']) # v2 모델
# plt.yticks(np.arange(0, 50, step = 5))
# plt.yticks(np.arange(0, 140000, step=10000))
# plt.yticks(np.arange(0, 60000, step=5000))
plt.yticks(np.arange(0, 10000, step=2000))
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

NameError: name 'plt' is not defined